In [ ]:
#!pip install pomegranate

In [ ]:
#!pip install pymc3

In [ ]:
#!pip install graphviz

In [ ]:
#!pip install pygraphviz

In [ ]:
#!pip install matplotlib

In [ ]:
#!pip install openpyxl

In [ ]:
#!pip install sklearn

In [45]:
#imports
import pandas as pd
import numpy as np
import arviz as az
import pymc3 as pm
import matplotlib.pyplot as plt
from pomegranate import *

In [46]:
#read in data
pd.set_option('display.max_columns', None)

#name format is df survey#_wave# if resend, add (_2 at end)

df1_1 = pd.read_csv("survey1.1.csv", encoding='cp1252')
df1_1_2 = pd.read_csv("survey1.1.2.csv", encoding='cp1252')
df1_2 = pd.read_csv("survey1.2.csv", encoding ='cp1252')
df1_3 = pd.read_csv("survey1.3.csv", encoding ='cp1252')
#survey 1 wave 3
#survey1.3.csv
#df1_3 = pd.read_csv...
df2_1 = pd.read_csv("survey2.1.csv", encoding='cp1252')
df2_1_2 = pd.read_csv("survey2.1.2.csv", encoding='cp1252')
df2_2 = pd.read_csv("survey2.2.csv", encoding='cp1252')
df2_3 = pd.read_csv("survey2.3.csv", encoding='cp1252')
df3_1 = pd.read_csv("survey3.1.csv", encoding='cp1252')
df3_2 = pd.read_csv("survey3.2.csv", encoding='cp1252')
print(df3_2.shape)

(13, 75)


In [9]:
display(df1_3.columns)

Index(['Date (Responses to recipient forms)', 'Recipient',
       'Origin (Responses to recipient forms)',
       'Score (Responses to recipient forms)', 'Primary key',
       'Caregiver Community Connectedness', 'MVCN Program Participation',
       'MVCN Volunteer', 'MVCN Support Programs and Services',
       'Continue Participation', 'Other Caregiver Engagement',
       'Caregive Support Programs Participation',
       'Conditional Question D or SD', 'Sense of Hope', 'Sense of Wellbeing',
       'Conditional: Portal Engagement Explanation', 'Depression',
       'Motivation', 'Life Satisfaction', 'Physical Exhaustion',
       'Desire To Escape', 'Same Self', 'Financial Decrease', 'Health Change',
       'Demands Strength', 'Competing Demand', 'Concern About Future',
       'Relationship Impact', 'Portal Experience', 'Portal Engagement',
       'Portal Engagement Reinforcement', 'Portal Wellbeing Influence',
       'Unhappy Alone', 'No One To Talk To', 'Cant Tolerate Being Alone',
   

In [10]:
df1_3


,Date (Responses to recipient forms),Recipient,Origin (Responses to recipient forms),Score (Responses to recipient forms),Primary key,Caregiver Community Connectedness,MVCN Program Participation,MVCN Volunteer,MVCN Support Programs and Services,Continue Participation,Other Caregiver Engagement,Caregive Support Programs Participation,Conditional Question D or SD,Sense of Hope,Sense of Wellbeing,Conditional: Portal Engagement Explanation,Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Hypertension (high blood pressure) (Chronic Conditions),Hyperlipidemia (high blood cholesterol or triglyceride levels) (Chronic Conditions),"Allergies, sinusitis and other upper respiratory conditions (Chronic Conditions)",Arthritis (Chronic Conditions),Mood Disorder (depression and bipolar disorder) (Chronic Conditions),Diabetes (Type1 and Type 2) (Chronic Conditions),Anxiety Disorder (Chronic Conditions),Asthma (Chronic Conditions),Coronary artery disease (includes myocardial infarction/heart attack) (Chronic Conditions),Thyroid disorder (Chronic Conditions),Chronic obstructive lung disease and bronchiectasis (Chronic Conditions),Traumatic Brain Injury (TBI) (Chronic Conditions),Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions),None of the above (Chronic Conditions),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,Consent
0,2021/06/03 17:45:55,Test Test (test@test.com),NaN,0,434484939,Agree,No,No,No,Neutral,Neutral,Neutral,NaN,Did NOT Particpiate,Did NOT Participate,NaN,No,No,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Agree,Agree,Agree,Agree,Sometimes,Rarely,Sometimes,Rarely,Rarely,Rarely,Rarely,Sometimes,Sometimes,Rarely,0,0,0,0,1,0,1,0,0,0,0,1,1,0,1,0,1,0,0,0,1,1,0,Agree,1
1,2021/06/04 11:43:34,Kules Nancy (nancykules@hotmail.com),NaN,0,434804578,Neutral,No,No,No,Neutral,Strongly Disagree,Strongly Disagree,"No time, do not plan to",Neutral,Neutral,NaN,No,Yes,Agree,Agree,Disagree,Neutral,Strongly Agree,Neutral,Neutral,Neutral,Disagree,Neutral,Agree,Neutral,Neutral,Neutral,Sometimes,Sometimes,Rarely,Rarely,Rarely,Rarely,Rarely,Sometimes,Never,Never,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,Disagree,1
2,2021/06/04 11:44:24,Mckelphin Rosalie (mckelphinfamily@gmail.com),NaN,0,434804579,Agree,Yes,No,Yes,Strongly Agree,Agree,Agree,NaN,Strongly Agree,Strongly Agree,NaN,No,No,Strongly Disagree,Agree,Agree,Agree,Agree,Disagree,Disagree,Disagree,Disagree,Agree,Agree,Agree,Agree,Agree,Never,Sometimes,Sometimes,Sometimes,Never,Sometimes,Sometimes,Never,Sometimes,Never,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,Neutral,1
3,2021/06/04 11:44:41,Bradley Kayla (kayla.bradley@me.com),NaN,0,434804580,Strongly Agree,Yes,No,Yes,Agree,Agree,Agree,NaN,Agree,Agree,NaN,Yes,Yes,Neutral,Agree,Agree,Disagree,Disagree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Sometimes,Sometimes,Rarely,Rarely,Rarely,Rarely,Rarely,Sometimes,Sometimes,Rarely,1,1,1,0,1,0,1,0,0,0,0,1,1,0,1,1,0,0,0,1,1,1,0,Strongly Agree,1
4,2021/06/04 11:46:17,Miller Melyssa (melyssamiller220@gmail.com),NaN,0,434804581,Agree,Unsure,No,Yes,Agree,Neutral,Neutral,NaN,Agree,Agree,NaN,No,No,Strongly Disagree,Disagree,Strongly Disagree,Disagree,Agree,Strongly Di

In [118]:
df1_3.shape

(309, 67)

In [35]:
print("shared\n")
for x in df2_3.columns:
    if x in df1_3.columns:
        print(x)
print("2.3\n")
for x in df2_3.columns:
   if not x in df1_3.columns:
        print('\'' + x + '\'' + ',')
print("1.3\n")
for x in df1_3.columns:
    if not x in df2_3.columns:
        print('\'' + x + '\'' + ',')

shared

Score (Responses to recipient forms)
Portal Chronic Disease Management Help
SMS/text (Connection Technology Use)
Phone (Connection Technology Use)
Video Portal (Connection Technology Use)
Other (Connection Technology Use)
Caregivers (Portal Video Use)
Family members (Portal Video Use)
Friends (Portal Video Use)
Others (Portal Video Use)
Not using Portal video (Portal Video Use)
Hypertension (high blood pressure) (Chronic Conditions)
Hyperlipidemia (high blood cholesterol or triglyceride levels)  (Chronic Conditions)
Allergies, sinusitis and other upper respiratory conditions  (Chronic Conditions)
Arthritis (Chronic Conditions)
Mood Disorder (depression and bipolar disorder) (Chronic Conditions)
Diabetes (Type1 and Type 2)  (Chronic Conditions)
Anxiety Disorder (Chronic Conditions)
Asthma (Chronic Conditions)
Coronary artery disease (includes myocardial infarction/heart attack)  (Chronic Conditions)
Thyroid disorder (Chronic Conditions)
Chronic obstructive lung disease and bronc

In [6]:
#print("shared\n")
#for x in df3_2.columns:
#    if x in df3_1.columns:
#        print(x)
#print("3.1\n")
#for x in df3_1.columns:
#    if not x in df3_2.columns:
#        print('\'' + x + '\'' + ',')
#print("3.2\n")
#for x in df3_2.columns:
#    if not x in df3_1.columns:
#        print('\'' + x + '\'' + ',')

In [47]:
def conditionalDrop(columns, df):
    for x in columns:
        if x in df.columns:
            df=df.drop(columns = x)
            
    return df

In [48]:
toDrop = ['Sense of Wellbeing',
'Sense of Hope',
'Conditional Question D or SD',
'Caregive Support Programs Participation',
'Other Caregiver Engagement',
'Continue Participation',
'MVCN Support Programs and Services',
'MVCN Volunteer',
'MVCN Program Participation',
'Caregiver Community Connectedness']
df3_2 = conditionalDrop(toDrop, df3_2)
df2_3 = conditionalDrop(toDrop, df2_3)
df1_3 = conditionalDrop(toDrop, df1_3)
toDrop = ['Video Conference (Connection Technology Use)',
'Race',
'Service 9/11',
'Spouse/Partner/Significant Other (Relationship)',
'Child (Relationship)',
'Sibling (Relationship)',
'Parent (Relationship)',
'Extended Family Member (Relationship)',
'Friend (Relationship)',
'Other (Relationship)',
'Work Status']
df3_2 = df3_2.drop(columns = toDrop)

In [49]:
survey1 =pd.concat([df1_1, df1_1_2, df1_2]) #add df1_3
survey2 = pd.concat([df2_1, df2_1_2, df2_2])
survey3 = pd.concat([df3_1])
survey1['survey']=np.full(survey1.shape[0], 1)
survey2['survey']=np.full(survey2.shape[0], 2)
survey3['survey']=np.full(survey3.shape[0], 3)

main = pd.concat([survey1, survey2])
main

,Date (Responses to recipient forms),Recipient,Origin (Responses to recipient forms),Score (Responses to recipient forms),Primary key,Conditional: Portal Engagement Explanation,Caregiver for Active Duty Military (Constituent Type),Caregiver for Veteran (Constituent Type),Other caregiver (Constituent Type),Veteran (Constituent Type),Other (Constituent Type),Sex,Age,Race:,Time Providing Care,Use of Video Conferencing,Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Hypertension (high blood pressure) (Chronic Conditions),Hyperlipidemia (high blood cholesterol or triglyceride levels) (Chronic Conditions),"Allergies, sinusitis and other upper respiratory conditions (Chronic Conditions)",Arthritis (Chronic Conditions),Mood Disorder (depression and bipolar disorder) (Chronic Conditions),Diabetes (Type1 and Type 2) (Chronic Conditions),Anxiety Disorder (Chronic Conditions),Asthma (Chronic Conditions),Coronary artery disease (includes myocardial infarction/heart attack) (Chronic Conditions),Thyroid disorder (Chronic Conditions),Chronic obstructive lung disease and bronchiectasis (Chronic Conditions),Traumatic Brain Injury (TBI) (Chronic Conditions),Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions),None of the above (Chronic Conditions),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,Consent,Flow,Display and Format,Content,survey
0,2020/12/04 10:55:54,Evans Melanie (mryumyumsbakery@gmail.com),NaN,15,397637986,NaN,0,1,0,0,0,Female,45 - 54,White,5+ years,1+ Year,Yes,No,Neutral,Agree,Disagree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Neutral,Strongly Disagree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Rarely,Sometimes,Never,Never,Sometimes,Sometimes,Rarely,Sometimes,Sometimes,Sometimes,1,0,0,1,0,0,1,1,0,1,0,0,0,0,1,1,1,0,0,1,1,1,0,Agree,1,5,5,5,1
1,2020/12/04 10:56:29,Miranda Jennifer (jmiranda26@icloud.com),NaN,14,397628088,NaN,0,1,0,0,0,Female,35 - 44,White,1 to 2 years,<3 Months,No,No,Strongly Disagree,Disagree,Strongly Disagree,Disagree,Disagree,Strongly Disagree,Strongly Disagree,Disagree,Strongly Disagree,Disagree,Agree,Agree,Agree,Neutral,Sometimes,Sometimes,Sometimes,Rarely,Never,Never,Never,Never,Rarely,Rarely,0,0,0,1,1,0,1,0,0,0,0,0,1,0,1,1,1,0,0,1,1,0,0,Agree,1,5,5,4,1
2,2020/12/04 10:56:42,Zuniga Adam (zuniga.adam77@gmail.com),NaN,15,397642487,NaN,0,0,0,1,0,Male,35 - 44,White,5+ years,3 To 6 Months,No,No,Strongly Disagree,Strongly Disagree,Strongly Disagree,Neutral,Strongly Agree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Neutral,Neutral,Agree,Agree,Never,Never,Rarely,Never,Never,Never,Never,Never,Never,Never,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,Agree,1,5,5,5,1
3,2020/12/04 10:57:02,Black Thomas (andrearuszala@gmail.com),NaN,12,397637989,NaN,0,0,0,1,0,Male,65+,White,6mo to 1year,6 To 9 Months,Yes,Yes,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Agree,Agree,Agree,Agree,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,Agree,1,4,4,4,1
4,2020/12/04 10:57:09,Jesseapontejr@Gmail.Com Jesse (jesseapontejr@g...,NaN,15,397642488,NaN,0,0,0,1,0,Male,35 - 44,Black or African American,2 to 3 years,1+ Year,

In [50]:
df2_3['survey']=np.full(df2_3.shape[0], 2)
df1_3['survey']=np.full(df1_3.shape[0], 1)

In [51]:
print(survey1.shape)
print(survey2.shape)
print(survey3.shape)
print(df1_3.shape)
print(df2_3.shape)

(702, 71)
(605, 71)
(12, 71)
(309, 58)
(312, 68)


In [52]:
toDrop = ['Time Providing Care','Use of Video Conferencing', 'Date (Responses to recipient forms)', 'Recipient', 'Origin (Responses to recipient forms)', 'Primary key', 'Conditional: Portal Engagement Explanation', 'Consent', 'Flow', 'Display and Format', 'Content',]
main = main.drop(columns = toDrop)
toDrop = ['Time Providing Care', 'Use of Video Conferencing','Date (Responses to recipient forms)', 'Recipient', 'Origin (Responses to recipient forms)', 'Primary key', 'Conditional: Portal Engagement Explanation', 'Consent',]
df2_3 = conditionalDrop(toDrop, df2_3)
df1_3 = conditionalDrop(toDrop, df1_3)
#main.drop(list(main.filter(regex = 'Constituent Type')), axis = 1, inplace = True)
#main.drop(list(main.filter(regex = 'Chronic Conditions')), axis = 1, inplace = True)
#main.drop(list(main.filter(regex = 'Portal Video Use')), axis = 1, inplace = True)
#main.drop(list(main.filter(regex = 'Connection Technology Use')), axis = 1, inplace = True)


In [53]:
df2_3.drop(list(main.filter(regex = 'Constituent Type')), axis = 1, inplace = True)
df2_3 = conditionalDrop(['Sex', 'Age', 'Race:',], df2_3) 

In [54]:
df2_3 = df2_3.dropna()
df1_3 = df1_3.dropna()

In [26]:
main

,Score (Responses to recipient forms),Caregiver for Active Duty Military (Constituent Type),Caregiver for Veteran (Constituent Type),Other caregiver (Constituent Type),Veteran (Constituent Type),Other (Constituent Type),Sex,Age,Race:,Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Hypertension (high blood pressure) (Chronic Conditions),Hyperlipidemia (high blood cholesterol or triglyceride levels) (Chronic Conditions),"Allergies, sinusitis and other upper respiratory conditions (Chronic Conditions)",Arthritis (Chronic Conditions),Mood Disorder (depression and bipolar disorder) (Chronic Conditions),Diabetes (Type1 and Type 2) (Chronic Conditions),Anxiety Disorder (Chronic Conditions),Asthma (Chronic Conditions),Coronary artery disease (includes myocardial infarction/heart attack) (Chronic Conditions),Thyroid disorder (Chronic Conditions),Chronic obstructive lung disease and bronchiectasis (Chronic Conditions),Traumatic Brain Injury (TBI) (Chronic Conditions),Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions),None of the above (Chronic Conditions),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,survey
0,15,0,1,0,0,0,Female,45 - 54,White,Yes,No,Neutral,Agree,Disagree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Neutral,Strongly Disagree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Rarely,Sometimes,Never,Never,Sometimes,Sometimes,Rarely,Sometimes,Sometimes,Sometimes,1,0,0,1,0,0,1,1,0,1,0,0,0,0,1,1,1,0,0,1,1,1,0,Agree,1
1,14,0,1,0,0,0,Female,35 - 44,White,No,No,Strongly Disagree,Disagree,Strongly Disagree,Disagree,Disagree,Strongly Disagree,Strongly Disagree,Disagree,Strongly Disagree,Disagree,Agree,Agree,Agree,Neutral,Sometimes,Sometimes,Sometimes,Rarely,Never,Never,Never,Never,Rarely,Rarely,0,0,0,1,1,0,1,0,0,0,0,0,1,0,1,1,1,0,0,1,1,0,0,Agree,1
2,15,0,0,0,1,0,Male,35 - 44,White,No,No,Strongly Disagree,Strongly Disagree,Strongly Disagree,Neutral,Strongly Agree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Neutral,Neutral,Agree,Agree,Never,Never,Rarely,Never,Never,Never,Never,Never,Never,Never,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,Agree,1
3,12,0,0,0,1,0,Male,65+,White,Yes,Yes,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Agree,Agree,Agree,Agree,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,Agree,1
4,15,0,0,0,1,0,Male,35 - 44,Black or African American,Yes,Yes,Strongly Disagree,Strongly Agree,Strongly Disagree,Neutral,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Neutral,Strongly Agree,Strongly Agree,Agree,Agree,Sometimes,Rarely,Sometimes,Sometimes,Rarely,Rarely,Rarely,Sometimes,Always,Rarely,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,1,0,Agree,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,15,0,1,0,0,0,Female,25 - 34,White,No,Yes,Strongly Disagree,Strongly Agree,Disagree,Agree,Neutral,Agree,Neutral,Strongly Disagree,Disagree,Agree,Neutral,Neutral,Agree,Agree,Rarel

In [36]:
df2_3

,Score (Responses to recipient forms),Portal Chronic Disease Management Help,SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),Hypertension (high blood pressure) (Chronic Conditions),Hyperlipidemia (high blood cholesterol or triglyceride levels) (Chronic Conditions),"Allergies, sinusitis and other upper respiratory conditions (Chronic Conditions)",Arthritis (Chronic Conditions),Mood Disorder (depression and bipolar disorder) (Chronic Conditions),Diabetes (Type1 and Type 2) (Chronic Conditions),Anxiety Disorder (Chronic Conditions),Asthma (Chronic Conditions),Coronary artery disease (includes myocardial infarction/heart attack) (Chronic Conditions),Thyroid disorder (Chronic Conditions),Chronic obstructive lung disease and bronchiectasis (Chronic Conditions),Traumatic Brain Injury (TBI) (Chronic Conditions),Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions),None of the above (Chronic Conditions),Feel Excluded,Difficult to make friends,Need Company,Unable To Communicate,Feel Completely Alone,Waiting for Contact,No One Understands Me,Cant Tolerate Being Alone,No One To Talk To,Unhappy Alone,Portal Wellbeing Influence,Portal Engagement Reinforcement,Portal Engagement,Portal Experience,Relationship Impact,Concern About Future,Competing Demand,Demands Strength,Health Change,Financial Decrease,Same Self,Desire To Escape,Physical Exhaustion,Life Satisfaction,Motivation,Depression,survey
0,0,Neutral,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,Sometimes,Always,Rarely,Rarely,Never,Sometimes,Sometimes,Rarely,Sometimes,Sometimes,Neutral,Neutral,Neutral,Neutral,Agree,Neutral,Neutral,Neutral,Disagree,Agree,Neutral,Strongly Disagree,Disagree,Disagree,No,No,2
1,0,Neutral,1,1,1,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,Sometimes,Always,Sometimes,Sometimes,Sometimes,Rarely,Sometimes,Sometimes,Sometimes,Sometimes,Agree,Agree,Strongly Agree,Strongly Agree,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Agree,Neutral,Agree,Disagree,Yes,Yes,2
2,0,Agree,1,1,1,0,0,1,1,0,0,0,0,1,1,1,0,1,0,0,0,0,1,1,0,Sometimes,Sometimes,Sometimes,Sometimes,Rarely,Sometimes,Sometimes,Rarely,Sometimes,Sometimes,Agree,Agree,Strongly Agree,Strongly Agree,Agree,Agree,Agree,Neutral,Neutral,Neutral,Agree,Strongly Disagree,Agree,Agree,Yes,Yes,2
3,0,Strongly Agree,1,1,1,0,1,1,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,Sometimes,Rarely,Rarely,Never,Rarely,Sometimes,Rarely,Rarely,Rarely,Rarely,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Neutral,Agree,Strongly Agree,Disagree,Neutral,Strongly Agree,Agree,Neutral,Agree,Neutral,Yes,Yes,2
4,0,Agree,0,1,1,0,0,1,1,0,0,1,1,0,1,1,0,0,0,0,0,0,0,1,0,Rarely,Rarely,Rarely,Sometimes,Sometimes,Rarely,Sometimes,Rarely,Sometimes,Sometimes,Agree,Agree,Agree,Agree,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Agree,Neutral,Agree,Agree,Yes,Yes,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,0,Strongly Agree,1,1,1,0,1,1,1,0,0,1,1,0,1,0,0,1,0,0,0,0,1,1,0,Sometimes,Sometimes,Rarely,Rarely,Sometimes,Sometimes,Rarely,Rarely,Sometimes,Sometimes,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Neutral,Strongly Agree,Neutral,Agree,Agree,Neutral,Strongly Disagree,Strongly Disagree,Neutral,Disagree,No,No,2
308,0,Strongly Disagree,1,1,0,0,0,1,0,0,0,1,1,1,1,1,0,1,1,0,0,0,0,1,0,Rarely,Always,Always,Always,Sometimes,Always,Sometimes,Sometimes,Sometimes,Always,Agree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Neutral,Strongly Agree,Agree,Strongly Agree,Neutral,Neutral,Agree,Agree,Yes,Yes,2
309,0,Neutral,0,0,0,1,0,1,0,0,0,1,1,1,1,0,0,1,1,0,1,0,0,0,0,Sometimes,Sometimes,Rarely,Sometimes,Sometimes,Rarely,

In [29]:
df1_3

,Score (Responses to recipient forms),Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Hypertension (high blood pressure) (Chronic Conditions),Hyperlipidemia (high blood cholesterol or triglyceride levels) (Chronic Conditions),"Allergies, sinusitis and other upper respiratory conditions (Chronic Conditions)",Arthritis (Chronic Conditions),Mood Disorder (depression and bipolar disorder) (Chronic Conditions),Diabetes (Type1 and Type 2) (Chronic Conditions),Anxiety Disorder (Chronic Conditions),Asthma (Chronic Conditions),Coronary artery disease (includes myocardial infarction/heart attack) (Chronic Conditions),Thyroid disorder (Chronic Conditions),Chronic obstructive lung disease and bronchiectasis (Chronic Conditions),Traumatic Brain Injury (TBI) (Chronic Conditions),Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions),None of the above (Chronic Conditions),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,survey
0,0,No,No,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Agree,Agree,Agree,Agree,Sometimes,Rarely,Sometimes,Rarely,Rarely,Rarely,Rarely,Sometimes,Sometimes,Rarely,0,0,0,0,1,0,1,0,0,0,0,1,1,0,1,0,1,0,0,0,1,1,0,Agree,2
1,0,No,Yes,Agree,Agree,Disagree,Neutral,Strongly Agree,Neutral,Neutral,Neutral,Disagree,Neutral,Agree,Neutral,Neutral,Neutral,Sometimes,Sometimes,Rarely,Rarely,Rarely,Rarely,Rarely,Sometimes,Never,Never,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,Disagree,2
2,0,No,No,Strongly Disagree,Agree,Agree,Agree,Agree,Disagree,Disagree,Disagree,Disagree,Agree,Agree,Agree,Agree,Agree,Never,Sometimes,Sometimes,Sometimes,Never,Sometimes,Sometimes,Never,Sometimes,Never,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,Neutral,2
3,0,Yes,Yes,Neutral,Agree,Agree,Disagree,Disagree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Sometimes,Sometimes,Rarely,Rarely,Rarely,Rarely,Rarely,Sometimes,Sometimes,Rarely,1,1,1,0,1,0,1,0,0,0,0,1,1,0,1,1,0,0,0,1,1,1,0,Strongly Agree,2
4,0,No,No,Strongly Disagree,Disagree,Strongly Disagree,Disagree,Agree,Strongly Disagree,Neutral,Disagree,Strongly Disagree,Disagree,Agree,Agree,Agree,Agree,Rarely,Never,Never,Never,Rarely,Never,Never,Never,Sometimes,Rarely,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,1,1,1,0,1,1,1,1,Agree,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,0,Yes,Yes,Strongly Disagree,Agree,Agree,Agree,Neutral,Disagree,Agree,Neutral,Neutral,Neutral,Agree,Agree,Agree,Agree,Rarely,Rarely,Rarely,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,1,1,0,0,1,1,1,0,Agree,2
305,0,No,No,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Agree,Strongly Agree,Strongly Agree,Agree,Never,Rarely,Never,Rarely,Never,Never,Never,Never,Never,Rarely,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,Agree,2
306,0,Yes,Yes,Strongly Disagree,Agree,Strongly Disagree,Agree,Neutral,Disagree,Neutral,Disagree,Strongly Disagree,Disagree,Disagree,Agree,Agree,Agree

In [55]:
encoding = {
    'Agree': 1,
    'StronglyAgree': 2,
    'Strongly Agree': 2,
    'StronglyDisagree': -2,
    'Strongly Disagree': -2,
    'Disagree': -1,
    'Neutral': 0,
    'Yes': 1,
    'Unsure': 0,
    'No': -1,
    'Rarely':-1,
    'Never':-2,
    'Sometimes':1,
    'Always':2,
    '<6mo':-3,
    '6mo to 1year':-2,
    '1 to 2 years': -1,
    '2 to 3 years': 0,
    '3 to 4 years':1,
    '4-5 years': 2,
    '5+ years': 3, 
    '<3 Months': -2,
    '3 To 6 Months': -1,
    '6 To 9 Months': 0,
    '9 To 12 Months': 1,
    '1+ Year': 2,
    '0 - 17': -3,
    '18 - 24': -2,
    '25 - 34': -1,
    '35 - 44': 0,
    '45 - 54': 1,
    '55 - 64': 2,
    '65+': 3,
    'Male': 0,
    'Female': 1,
    'White': -2, 
    'Black or African American': -1,
    'Asian': 0,
    'American Indian or Alaska Native': 1,
    'Some other race': 2,
    'Native Hawaiian and Pacific Islander': 3,
    'Did NOT Participate': -10,
    'Did NOT Particpiate': -10
}
def f(s):
    l = []
    for b in s:
        #print(b)
        if (b in encoding):
            l.append(encoding[b])
        elif isinstance(b, str) and b.isnumeric():
            l.append(int(b))
        elif np.isnan(b):
            l.append(int(-9))
        else:
            l.append(b)
    return pd.Series(l)


In [82]:
data = main.apply(f, axis = 0)
data = data.sample(frac=1).reset_index(drop=True)
data
data2_3 = df2_3.apply(f, axis = 0)
data1_3 = df1_3.apply(f, axis = 0)
data2_3 =data2_3.sample(frac=1).reset_index(drop=True)
data1_3 =data1_3.sample(frac=1).reset_index(drop=True)

In [70]:
data

,Score (Responses to recipient forms),Caregiver for Active Duty Military (Constituent Type),Caregiver for Veteran (Constituent Type),Other caregiver (Constituent Type),Veteran (Constituent Type),Other (Constituent Type),Sex,Age,Race:,Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Hypertension (high blood pressure) (Chronic Conditions),Hyperlipidemia (high blood cholesterol or triglyceride levels) (Chronic Conditions),"Allergies, sinusitis and other upper respiratory conditions (Chronic Conditions)",Arthritis (Chronic Conditions),Mood Disorder (depression and bipolar disorder) (Chronic Conditions),Diabetes (Type1 and Type 2) (Chronic Conditions),Anxiety Disorder (Chronic Conditions),Asthma (Chronic Conditions),Coronary artery disease (includes myocardial infarction/heart attack) (Chronic Conditions),Thyroid disorder (Chronic Conditions),Chronic obstructive lung disease and bronchiectasis (Chronic Conditions),Traumatic Brain Injury (TBI) (Chronic Conditions),Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions),None of the above (Chronic Conditions),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,survey
0,11,0,0,0,1,0,0,1,-2,1,1,-2,1,1,2,1,-1,1,1,1,0,1,1,0,0,1,2,2,2,1,1,1,1,2,2,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,1,1,1,0,1,0,2
1,15,0,1,0,0,0,1,2,-2,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,2
2,10,0,1,0,0,0,1,1,-2,-1,1,0,1,0,1,0,1,1,1,0,-1,1,1,1,1,1,1,1,2,1,1,1,1,1,-2,0,1,1,1,1,0,1,1,0,1,1,0,1,0,0,1,0,0,0,0,0,1,0,1,2
3,15,0,0,0,1,0,0,0,-2,1,1,2,2,2,2,0,2,2,2,2,2,2,2,2,2,1,1,2,2,2,1,2,2,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,1,0,0,1,1,1,0,2,1
4,15,0,0,0,1,0,0,2,-2,-1,-1,0,1,-2,0,-1,-1,1,-1,-1,-2,2,1,1,1,-1,-2,-2,-2,-1,-2,-2,-2,-2,-2,1,1,0,0,0,0,1,0,1,0,0,0,1,0,1,1,1,0,0,1,1,1,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,10,0,0,0,1,0,0,0,-2,1,1,1,2,1,1,2,0,2,1,2,2,0,0,0,0,2,1,1,1,2,-1,1,2,2,1,0,1,1,1,1,0,1,0,0,1,1,1,1,0,0,0,0,1,0,1,1,0,0,0,2
1303,15,0,1,0,0,0,1,2,-2,1,1,1,1,1,2,2,-1,1,-1,1,1,2,2,1,1,2,1,1,-1,-1,1,1,1,-1,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,1,0,0,0,0,0,1,1,2
1304,15,0,1,0,0,0,1,2,2,-1,-1,0,0,-1,0,1,0,0,0,0,0,1,2,2,2,-1,1,1,1,1,1,1,1,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,1,1,0,2,2
1305,15,0,1,0,0,0,1,-1,-2,-1,-1,-2,-2,-2,-2,0,-2,-1,-2,-2,-2,2,2,1,1,1,-2,-1,-2,-1,-2,-2,-1,1,-1,1,1,1,1,0,0,0,1,0,1,0,1,0,0,0,1,1,0,0,0,0,0,1,1,1


In [71]:
data1_3

,Score (Responses to recipient forms),Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Hypertension (high blood pressure) (Chronic Conditions),Hyperlipidemia (high blood cholesterol or triglyceride levels) (Chronic Conditions),"Allergies, sinusitis and other upper respiratory conditions (Chronic Conditions)",Arthritis (Chronic Conditions),Mood Disorder (depression and bipolar disorder) (Chronic Conditions),Diabetes (Type1 and Type 2) (Chronic Conditions),Anxiety Disorder (Chronic Conditions),Asthma (Chronic Conditions),Coronary artery disease (includes myocardial infarction/heart attack) (Chronic Conditions),Thyroid disorder (Chronic Conditions),Chronic obstructive lung disease and bronchiectasis (Chronic Conditions),Traumatic Brain Injury (TBI) (Chronic Conditions),Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions),None of the above (Chronic Conditions),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,survey
0,0,1,1,-1,1,0,1,0,-1,1,1,-2,1,1,1,1,1,1,1,-1,1,1,1,1,-1,1,-1,0,0,1,1,1,0,1,0,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1
1,0,-1,1,0,0,0,0,0,0,0,0,0,0,2,2,1,1,1,-1,-2,1,-2,-2,-2,-2,-2,-2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,2,1
2,0,-1,-1,-2,-2,-2,-2,-2,-1,1,-2,-2,0,1,1,1,1,-2,1,-1,-1,1,-1,-2,1,-1,-2,1,1,1,1,0,0,0,1,0,0,0,1,1,0,0,1,0,1,0,0,0,1,0,1,1
3,0,1,-1,-2,1,-1,-2,1,-2,-2,0,0,0,1,1,1,1,1,1,-1,1,-1,-1,-1,-2,-1,1,1,0,1,0,1,0,1,1,0,0,0,1,1,0,1,1,1,0,0,1,1,1,0,2,1
4,0,1,1,1,2,2,1,0,1,1,1,0,0,1,1,1,1,1,1,1,1,1,-1,1,-1,2,-1,0,1,1,1,1,1,1,0,0,0,0,0,1,0,1,1,1,0,0,1,1,1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,0,1,1,1,2,2,2,1,1,2,1,1,1,1,0,0,0,1,1,-1,1,1,-1,-1,-2,2,1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,1
305,0,-1,1,-1,0,-1,1,2,0,-1,1,2,0,2,0,2,1,2,1,-1,1,-1,-2,-2,-1,1,2,0,0,1,0,1,1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1
306,0,1,1,-1,1,0,1,-1,-1,0,0,0,0,0,0,0,0,1,-1,-2,-1,-1,-2,-1,-1,2,-1,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,1,1,0,0,1,1,1,1,1,1
307,0,1,1,2,2,2,2,0,2,2,2,0,0,-1,0,-2,-2,2,2,1,2,1,1,1,1,2,2,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,-1,1


In [18]:
#ls = []
#for column in data:
#    ls.append(data[column].unique())
#ls

In [19]:
#data1 = data[data['survey']==1]
#data2 = data[data['survey']==2]

In [20]:
#data1.sum(axis=0)/702

In [21]:
pd.set_option('display.max_rows', 100)

In [83]:
temp = data.copy()
temp2_3 = data2_3.copy()
temp1_3= data1_3.copy()
temp.drop(list(main.filter(regex = 'Chronic Conditions')), axis = 1, inplace = True)
temp.drop(list(main.filter(regex = 'Constituent Type')), axis = 1, inplace = True)
temp = temp.drop(columns = ['Sex', 'Age', 'Race:',])
temp2_3.drop(list(main.filter(regex = 'Chronic Conditions')), axis = 1, inplace = True)
temp1_3.drop(list(main.filter(regex = 'Chronic Conditions')), axis = 1, inplace = True)
temp

,Score (Responses to recipient forms),Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,survey
0,11,1,-1,-2,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,2,2,0,1,0,0,0,1,1,0,1,0,2
1,15,1,-1,-1,2,-2,-1,0,0,1,1,0,1,1,1,1,0,1,1,-1,1,2,1,2,1,2,2,0,1,0,0,0,1,1,1,0,0,2
2,12,-1,-1,2,1,-2,0,0,1,0,-1,-1,-2,1,2,2,1,-1,-1,1,-1,-1,-2,-2,-2,-2,-2,1,0,1,1,0,0,1,0,0,2,2
3,15,1,1,1,2,2,2,0,0,1,-2,0,1,2,1,0,0,1,-1,-1,1,1,-1,-2,-1,2,-1,0,0,0,0,1,0,0,0,1,0,1
4,15,1,1,1,1,2,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,2,1,1,1,1,0,0,0,1,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,12,1,1,1,2,2,2,-2,1,1,0,1,0,0,0,1,1,1,1,-1,1,1,1,1,1,2,1,0,0,0,0,1,0,1,0,0,0,2
1303,12,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,-1,-1,1,1,-2,-1,-1,-1,0,0,0,0,1,0,0,0,1,2,1
1304,13,1,1,0,1,0,0,1,0,1,0,-1,1,0,1,0,0,1,1,1,1,2,1,1,1,2,1,1,1,1,0,0,1,1,1,0,1,2
1305,9,-1,-1,0,1,1,1,2,1,1,0,0,0,1,2,0,0,1,1,-1,1,-1,1,1,1,-1,1,0,1,1,0,0,1,1,0,1,0,1


In [61]:
hred = ['Portal Experience', 'Portal Engagement', 'Portal Engagement Reinforcement', 'Portal Wellbeing Influence', 'Use of Video Conferencing', 'Caregivers (Portal Video Use)', 'Family members (Portal Video Use)','Friends (Portal Video Use)', 'Others (Portal Video Use)', 'SMS/text (Connection Technology Use)','Phone (Connection Technology Use)','Video Portal (Connection Technology Use)','Other (Connection Technology Use)','Portal Chronic Disease Management Help'] 

In [62]:
def custom_style(row):

    color = ''
    if row.values[0] in hred and row.values[-1] == "Alternative accepted: mean is smaller":
        color = 'lime'
    if (not row.values[0] in hred) and row.values[-1] == "Alternative accepted: mean is larger":
        color = 'lime'
    if (not row.values[0] in hred) and row.values[-1] == "Alternative accepted: mean is smaller":
        color = 'red'
    if row.values[0] in hred and row.values[-1] == "Alternative accepted: mean is larger":
        color = 'red'

    return ['background-color: %s' % color]*len(row.values)

In [73]:
def stats(data1, data2):
    results = []

    for col in data1.columns: 
        vals1 = data1[col].to_list()
        vals2 = data2[col].to_list()
    
    # Null hypothesis: means are the same
    # Alternative: the mean after is higher
    
        cm = ssw.CompareMeans(ssw.DescrStatsW(vals1), ssw.DescrStatsW(vals2))
        res = cm.ztest_ind(alternative='smaller',usevar='unequal')
        msg = "none"
        alpha = 0.05/39
        if (res[1] <= alpha):
            msg = "larger"
       
        elif (res[1] >= 1-alpha):
            msg = "smaller"
    
        if (res[1] <= 0.5):
            results.append([col, res[1], msg])
        else:
            results.append([col, 1-res[1], msg])

    res = pd.DataFrame(results, columns=['Item', 'P-Value', "difference"])
    return res #res.style.apply(custom_style, axis=1)

In [74]:
w1data = temp.loc[data['survey']==1]
w2data = temp.loc[data['survey']==2]

import statsmodels.stats.weightstats as ssw
display(stats(w1data, w2data)[:19])
stats(w1data, w2data)[19:]


,Item,P-Value,difference
0,Score (Responses to recipient forms),7.534456e-12,larger
1,Depression,4.022708e-01,none
2,Motivation,2.262188e-01,none
3,Life Satisfaction,2.104960e-03,none
4,Physical Exhaustion,2.088118e-02,none
5,Desire To Escape,5.074631e-02,none
6,Same Self,9.662410e-02,none
7,Financial Decrease,3.121028e-02,none
8,Health Change,1.291668e-03,none
9,Demands Strength,9.723762e-05,larger


,Item,P-Value,difference
19,Cant Tolerate Being Alone,4.521131e-01,none
20,No One Understands Me,4.191203e-01,none
21,Waiting for Contact,7.861060e-03,none
22,Feel Completely Alone,1.368493e-01,none
23,Unable To Communicate,6.235237e-03,none
24,Need Company,1.123059e-02,none
25,Difficult to make friends,3.018716e-01,none
26,Feel Excluded,1.303657e-03,none
27,Caregivers (Portal Video Use),3.022412e-01,none
28,Family members (Portal Video Use),4.420908e-13,smaller


In [75]:
def stat3(turn, c, temp, data):
    print(c)
    a1data = temp[data[c]<= turn]
    a2data = temp[data[c]>= turn+1]

    display(stats(a1data, a2data)[:19])
    display(stats(a1data, a2data)[19:])
    
    b1data = a1data.loc[(data['survey'] == 1)]
    b2data = a1data.loc[(data['survey'] == 2)]
 
    display(stats(b1data, b2data)[:19])
    display(stats(b1data, b2data)[19:])
    
    c1data = a2data.loc[(data['survey'] == 1)]
    c2data = a2data.loc[(data['survey'] == 2)]

    display(stats(c1data, c2data)[:19])
    display(stats(c1data, c2data)[19:])

In [88]:
pd.set_option('display.float_format', lambda x: '{:e}'.format(x))
ndata = pd.concat([data, data2_3, data1_3], ignore_index=True)
ntemp = pd.concat([temp, temp2_3, temp1_3], ignore_index=True)

In [89]:
ntemp[ntemp.index.duplicated()]
ntemp


,Score (Responses to recipient forms),Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,survey
0,11,1,-1,-2,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,2,2,0,1,0,0,0,1,1,0,1,0,2
1,15,1,-1,-1,2,-2,-1,0,0,1,1,0,1,1,1,1,0,1,1,-1,1,2,1,2,1,2,2,0,1,0,0,0,1,1,1,0,0,2
2,12,-1,-1,2,1,-2,0,0,1,0,-1,-1,-2,1,2,2,1,-1,-1,1,-1,-1,-2,-2,-2,-2,-2,1,0,1,1,0,0,1,0,0,2,2
3,15,1,1,1,2,2,2,0,0,1,-2,0,1,2,1,0,0,1,-1,-1,1,1,-1,-2,-1,2,-1,0,0,0,0,1,0,0,0,1,0,1
4,15,1,1,1,1,2,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,2,1,1,1,1,0,0,0,1,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1923,0,-1,-1,-2,-2,-2,-2,-1,-2,1,-2,-2,-2,1,1,2,2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,1,0,0,0,0,1,1,1,0,2,1
1924,0,1,1,1,1,1,2,0,1,1,2,1,2,0,0,1,1,1,1,1,1,-1,1,1,1,1,1,0,1,0,0,0,1,1,1,0,1,1
1925,0,1,1,-1,0,0,-1,1,-1,0,0,0,0,0,0,0,1,-1,1,-1,-1,-1,-1,1,-1,-2,-2,1,0,0,0,0,1,0,1,0,1,1
1926,0,1,1,0,2,1,1,-1,1,2,0,1,1,1,0,1,0,1,1,1,1,1,1,1,-1,1,-1,1,1,1,0,0,1,0,1,0,1,1


In [90]:
thing = ['Hypertension (high blood pressure) (Chronic Conditions)','Hyperlipidemia (high blood cholesterol or triglyceride levels)  (Chronic Conditions)','Allergies, sinusitis and other upper respiratory conditions  (Chronic Conditions)','Arthritis (Chronic Conditions)','Diabetes (Type1 and Type 2)  (Chronic Conditions)', 'Mood Disorder (depression and bipolar disorder) (Chronic Conditions)','Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions)','Anxiety Disorder (Chronic Conditions)','Asthma (Chronic Conditions)','Thyroid disorder (Chronic Conditions)','Traumatic Brain Injury (TBI) (Chronic Conditions)']
choice = thing[1] #change this to change what we're analysing
choice = 'Traumatic Brain Injury (TBI) (Chronic Conditions)'

stat3(0, 'Age', temp, data)

stat3(0, 'Sex', temp, data)
stat3(-2, 'Race:', temp, data)

stat3(0, 'Veteran (Constituent Type)', temp, data)



for choice in thing:
    stat3(0, choice, ntemp, ndata)


Age


,Item,P-Value,difference
0,Score (Responses to recipient forms),6.407840e-02,none
1,Depression,6.282936e-02,none
2,Motivation,1.526732e-02,none
3,Life Satisfaction,1.056282e-01,none
4,Physical Exhaustion,1.273992e-01,none
5,Desire To Escape,6.150418e-02,none
6,Same Self,2.171235e-01,none
7,Financial Decrease,3.370092e-01,none
8,Health Change,6.756348e-03,none
9,Demands Strength,3.783779e-04,larger


,Item,P-Value,difference
19,Cant Tolerate Being Alone,1.367911e-01,none
20,No One Understands Me,4.774071e-03,none
21,Waiting for Contact,1.358000e-02,none
22,Feel Completely Alone,5.021666e-02,none
23,Unable To Communicate,2.865221e-02,none
24,Need Company,1.811257e-01,none
25,Difficult to make friends,6.965507e-07,smaller
26,Feel Excluded,4.499071e-05,smaller
27,Caregivers (Portal Video Use),4.586704e-05,smaller
28,Family members (Portal Video Use),4.514474e-06,smaller


,Item,P-Value,difference
0,Score (Responses to recipient forms),5.791772e-05,larger
1,Depression,1.899161e-01,none
2,Motivation,2.652056e-01,none
3,Life Satisfaction,1.088902e-05,larger
4,Physical Exhaustion,2.371919e-06,larger
5,Desire To Escape,2.338020e-03,none
6,Same Self,5.165889e-05,larger
7,Financial Decrease,1.270707e-03,larger
8,Health Change,2.710560e-05,larger
9,Demands Strength,5.011834e-08,larger


,Item,P-Value,difference
19,Cant Tolerate Being Alone,4.884786e-01,none
20,No One Understands Me,4.698195e-02,none
21,Waiting for Contact,3.636847e-02,none
22,Feel Completely Alone,1.137981e-01,none
23,Unable To Communicate,1.665792e-03,none
24,Need Company,2.470463e-02,none
25,Difficult to make friends,6.557811e-03,none
26,Feel Excluded,6.676543e-05,larger
27,Caregivers (Portal Video Use),2.662424e-01,none
28,Family members (Portal Video Use),2.085428e-05,smaller


...

,Item,P-Value,difference
19,Cant Tolerate Being Alone,2.083177e-02,none
20,No One Understands Me,5.261618e-03,none
21,Waiting for Contact,6.995293e-02,none
22,Feel Completely Alone,2.273120e-01,none
23,Unable To Communicate,9.696787e-02,none
24,Need Company,2.749096e-02,none
25,Difficult to make friends,1.560033e-07,larger
26,Feel Excluded,8.520638e-05,larger
27,Caregivers (Portal Video Use),7.147732e-02,none
28,Family members (Portal Video Use),1.429016e-02,none


,Item,P-Value,difference
0,Score (Responses to recipient forms),4.793632e-01,none
1,Depression,2.413754e-01,none
2,Motivation,2.380158e-01,none
3,Life Satisfaction,7.697625e-03,none
4,Physical Exhaustion,4.673445e-01,none
5,Desire To Escape,2.245563e-01,none
6,Same Self,4.174183e-01,none
7,Financial Decrease,4.325008e-01,none
8,Health Change,3.512350e-02,none
9,Demands Strength,2.984499e-03,none


,Item,P-Value,difference
19,Cant Tolerate Being Alone,3.630353e-01,none
20,No One Understands Me,3.427121e-02,none
21,Waiting for Contact,2.284071e-01,none
22,Feel Completely Alone,1.365177e-01,none
23,Unable To Communicate,1.775908e-01,none
24,Need Company,1.489207e-02,none
25,Difficult to make friends,2.211613e-01,none
26,Feel Excluded,8.587019e-02,none
27,Caregivers (Portal Video Use),5.027780e-02,none
28,Family members (Portal Video Use),7.802003e-08,smaller


,Item,P-Value,difference
0,Score (Responses to recipient forms),2.484799e-01,none
1,Depression,4.795462e-01,none
2,Motivation,3.915009e-01,none
3,Life Satisfaction,3.875018e-03,none
4,Physical Exhaustion,9.874666e-03,none
5,Desire To Escape,3.159886e-01,none
6,Same Self,1.883166e-02,none
7,Financial Decrease,1.485458e-02,none
8,Health Change,1.481028e-04,larger
9,Demands Strength,1.273385e-04,larger


,Item,P-Value,difference
19,Cant Tolerate Being Alone,2.877760e-01,none
20,No One Understands Me,7.602116e-02,none
21,Waiting for Contact,2.011818e-02,none
22,Feel Completely Alone,2.758756e-01,none
23,Unable To Communicate,3.228076e-02,none
24,Need Company,6.588756e-03,none
25,Difficult to make friends,2.498706e-02,none
26,Feel Excluded,5.145705e-04,larger
27,Caregivers (Portal Video Use),4.084819e-01,none
28,Family members (Portal Video Use),1.120132e-02,none


In [119]:
stat3(0, 'Chronic obstructive lung disease and bronchiectasis  (Chronic Conditions)', ntemp, ndata)

Chronic obstructive lung disease and bronchiectasis  (Chronic Conditions)


,Item,P-Value,difference
0,Score (Responses to recipient forms),3.732759e-01,none
1,Depression,2.082879e-01,none
2,Motivation,9.322206e-05,larger
3,Life Satisfaction,1.940650e-01,none
4,Physical Exhaustion,3.852313e-04,larger
5,Desire To Escape,3.279298e-01,none
6,Same Self,4.337731e-03,none
7,Financial Decrease,2.180873e-01,none
8,Health Change,1.885230e-01,none
9,Demands Strength,9.900825e-02,none


,Item,P-Value,difference
19,Cant Tolerate Being Alone,4.860615e-02,none
20,No One Understands Me,2.718706e-01,none
21,Waiting for Contact,3.494442e-02,none
22,Feel Completely Alone,8.248441e-02,none
23,Unable To Communicate,4.687921e-01,none
24,Need Company,4.382960e-02,none
25,Difficult to make friends,1.640360e-01,none
26,Feel Excluded,1.975380e-01,none
27,Caregivers (Portal Video Use),6.516258e-02,none
28,Family members (Portal Video Use),1.606331e-01,none


,Item,P-Value,difference
0,Score (Responses to recipient forms),3.219834e-01,none
1,Depression,1.418820e-01,none
2,Motivation,3.898886e-01,none
3,Life Satisfaction,4.860941e-04,larger
4,Physical Exhaustion,8.419286e-02,none
5,Desire To Escape,5.167491e-02,none
6,Same Self,1.139063e-01,none
7,Financial Decrease,8.923424e-02,none
8,Health Change,8.936350e-04,larger
9,Demands Strength,7.640992e-05,larger


,Item,P-Value,difference
19,Cant Tolerate Being Alone,3.219391e-01,none
20,No One Understands Me,4.902791e-01,none
21,Waiting for Contact,3.352315e-02,none
22,Feel Completely Alone,4.642959e-02,none
23,Unable To Communicate,2.480292e-02,none
24,Need Company,6.358810e-04,larger
25,Difficult to make friends,4.909419e-01,none
26,Feel Excluded,3.415206e-03,none
27,Caregivers (Portal Video Use),5.051351e-02,none
28,Family members (Portal Video Use),5.124395e-08,smaller


,Item,P-Value,difference
0,Score (Responses to recipient forms),2.965104e-01,none
1,Depression,7.739538e-03,none
2,Motivation,5.588103e-04,smaller
3,Life Satisfaction,2.518558e-01,none
4,Physical Exhaustion,3.139290e-01,none
5,Desire To Escape,8.634297e-03,none
6,Same Self,4.176392e-02,none
7,Financial Decrease,4.572919e-01,none
8,Health Change,4.647779e-02,none
9,Demands Strength,6.957664e-03,none


,Item,P-Value,difference
19,Cant Tolerate Being Alone,4.175049e-01,none
20,No One Understands Me,1.171551e-03,smaller
21,Waiting for Contact,3.573991e-01,none
22,Feel Completely Alone,8.858079e-02,none
23,Unable To Communicate,3.146334e-01,none
24,Need Company,4.213008e-01,none
25,Difficult to make friends,4.696929e-01,none
26,Feel Excluded,4.177073e-01,none
27,Caregivers (Portal Video Use),4.821137e-01,none
28,Family members (Portal Video Use),1.502512e-02,none


In [ ]:
ndata.

In [91]:
test1 = ndata[ndata['survey']==1]
test2 = ndata[ndata['survey']==2]
l1 = test1.shape[0]
l2 = test2.shape[0]
print(l1)
print(l2)
diseases = ['Hypertension (high blood pressure) (Chronic Conditions)','Hyperlipidemia (high blood cholesterol or triglyceride levels)  (Chronic Conditions)','Allergies, sinusitis and other upper respiratory conditions  (Chronic Conditions)','Arthritis (Chronic Conditions)','Mood Disorder (depression and bipolar disorder) (Chronic Conditions)','Diabetes (Type1 and Type 2)  (Chronic Conditions)','Anxiety Disorder (Chronic Conditions)','Asthma (Chronic Conditions)','Coronary artery disease (includes myocardial infarction/heart attack)  (Chronic Conditions)','Thyroid disorder (Chronic Conditions)','Chronic obstructive lung disease and bronchiectasis  (Chronic Conditions)','Traumatic Brain Injury (TBI) (Chronic Conditions)','Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions)','None of the above (Chronic Conditions)']
caregiving = ['Caregiver for Active Duty Military (Constituent Type)','Caregiver for Veteran (Constituent Type)','Other caregiver (Constituent Type)','Veteran (Constituent Type)','Other (Constituent Type)']
for x in diseases:
    a = test1[test1[x]==1].shape[0]
    b = test2[test2[x]==1].shape[0]
    print(f'{x:88}  {a:6} {float(a)*100./l1:.2f} {b:6} {float(b)*100./l2:.2f}')


1011
917
Hypertension (high blood pressure) (Chronic Conditions)                                      418 41.35    496 54.09
Hyperlipidemia (high blood cholesterol or triglyceride levels)  (Chronic Conditions)         255 25.22    323 35.22
Allergies, sinusitis and other upper respiratory conditions  (Chronic Conditions)            558 55.19    462 50.38
Arthritis (Chronic Conditions)                                                               468 46.29    496 54.09
Mood Disorder (depression and bipolar disorder) (Chronic Conditions)                         616 60.93    519 56.60
Diabetes (Type1 and Type 2)  (Chronic Conditions)                                            226 22.35    231 25.19
Anxiety Disorder (Chronic Conditions)                                                        603 59.64    493 53.76
Asthma (Chronic Conditions)                                                                  236 23.34    178 19.41
Coronary artery disease (includes myocardial infarction/heart a

In [92]:
w1data = ntemp.loc[ndata['survey']==1]
w2data = ntemp.loc[ndata['survey']==2]

display(stats(w1data, w2data)[:19])
stats(w1data, w2data)[19:]


,Item,P-Value,difference
0,Score (Responses to recipient forms),2.715634e-01,none
1,Depression,4.121892e-01,none
2,Motivation,1.173350e-01,none
3,Life Satisfaction,4.756755e-04,larger
4,Physical Exhaustion,1.091406e-01,none
5,Desire To Escape,2.099043e-01,none
6,Same Self,2.460749e-01,none
7,Financial Decrease,9.280687e-02,none
8,Health Change,2.268233e-04,larger
9,Demands Strength,5.122794e-06,larger


,Item,P-Value,difference
19,Cant Tolerate Being Alone,3.418943e-01,none
20,No One Understands Me,1.801396e-01,none
21,Waiting for Contact,4.988309e-02,none
22,Feel Completely Alone,1.235868e-01,none
23,Unable To Communicate,4.534222e-02,none
24,Need Company,1.344852e-03,none
25,Difficult to make friends,4.886824e-01,none
26,Feel Excluded,4.592385e-03,none
27,Caregivers (Portal Video Use),5.515901e-02,none
28,Family members (Portal Video Use),3.987060e-09,smaller


In [102]:
df2_3 = pd.read_csv("survey2.3.csv", encoding='cp1252')
df1_3 = pd.read_csv("survey1.3.csv", encoding='cp1252')
tdf = df2_3.copy()
sdf = df1_3.copy()

In [94]:
tdf

,Date (Responses to recipient forms),Recipient,Origin (Responses to recipient forms),Score (Responses to recipient forms),Primary key,Consent,Portal Chronic Disease Management Help,SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),Hypertension (high blood pressure) (Chronic Conditions),Hyperlipidemia (high blood cholesterol or triglyceride levels) (Chronic Conditions),"Allergies, sinusitis and other upper respiratory conditions (Chronic Conditions)",Arthritis (Chronic Conditions),Mood Disorder (depression and bipolar disorder) (Chronic Conditions),Diabetes (Type1 and Type 2) (Chronic Conditions),Anxiety Disorder (Chronic Conditions),Asthma (Chronic Conditions),Coronary artery disease (includes myocardial infarction/heart attack) (Chronic Conditions),Thyroid disorder (Chronic Conditions),Chronic obstructive lung disease and bronchiectasis (Chronic Conditions),Traumatic Brain Injury (TBI) (Chronic Conditions),Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions),None of the above (Chronic Conditions),Feel Excluded,Difficult to make friends,Need Company,Unable To Communicate,Feel Completely Alone,Waiting for Contact,No One Understands Me,Cant Tolerate Being Alone,No One To Talk To,Unhappy Alone,Portal Wellbeing Influence,Portal Engagement Reinforcement,Portal Engagement,Portal Experience,Relationship Impact,Concern About Future,Competing Demand,Demands Strength,Health Change,Financial Decrease,Same Self,Desire To Escape,Physical Exhaustion,Life Satisfaction,Motivation,Depression,Use of Video Conferencing,Time Providing Care,Race:,Age,Sex,Caregiver for Active Duty Military (Constituent Type),Caregiver for Veteran (Constituent Type),Other caregiver (Constituent Type),Veteran (Constituent Type),Other (Constituent Type),Conditional: Portal Engagement Explanation,Sense of Wellbeing,Sense of Hope,Conditional Question D or SD,Caregive Support Programs Participation,Other Caregiver Engagement,Continue Participation,MVCN Support Programs and Services,MVCN Volunteer,MVCN Program Participation,Caregiver Community Connectedness
0,5/4/2021 15:54,Shepherd Farrah (ftshepherd@gmail.com),nan,0,425837836,1,Neutral,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,Sometimes,Always,Rarely,Rarely,Never,Sometimes,Sometimes,Rarely,Sometimes,Sometimes,Neutral,Neutral,Neutral,Neutral,Agree,Neutral,Neutral,Neutral,Disagree,Agree,Neutral,Strongly Disagree,Disagree,Disagree,No,No,nan,nan,nan,nan,nan,0,0,0,0,0,NaN,Agree,Agree,NaN,Agree,Agree,Agree,Unsure,No,Unsure,Neutral
1,5/4/2021 15:56,Smith Candice (mrssmith0429@gmail.com),nan,0,425844995,1,Neutral,1,1,1,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,Sometimes,Always,Sometimes,Sometimes,Sometimes,Rarely,Sometimes,Sometimes,Sometimes,Sometimes,Agree,Agree,Strongly Agree,Strongly Agree,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Agree,Neutral,Agree,Disagree,Yes,Yes,nan,nan,nan,nan,nan,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5/4/2021 15:56,Messer Jessica (crash2183@hotmail.com),nan,0,425848900,1,Agree,1,1,1,0,0,1,1,0,0,0,0,1,1,1,0,1,0,0,0,0,1,1,0,Sometimes,Sometimes,Sometimes,Sometimes,Rarely,Sometimes,Sometimes,Rarely,Sometimes,Sometimes,Agree,Agree,Strongly Agree,Strongly Agree,Agree,Agree,Agree,Neutral,Neutral,Neutral,Agree,Strongly Disagree,Agree,Agree,Yes,Yes,nan,nan,nan,nan,nan,0,0,0,0,0,NaN,Agree,Agree,NaN,Agree,Neutral,Agree,Unsure,No,Unsure,Agree
3,5/4/2021 15:56,Gtsbsahleen@Gmail.Com Tracy (gtsbsahleen@gmail...,nan,0,425848901,1,Strongly Agree,1,1,1,0,1,1,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,Sometimes,Rarely,Rarely,Never,Rarely,Sometimes,Rarely,Rarely,Rarely,Rarely,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Neutral,Agree,Strongly Agree,Disagree,Neutral,Strongly Agree,Agree,Neutral,Agree,Neutral,Yes,Yes,nan,nan,nan

In [103]:
toDrop = [
'Conditional Question D or SD',
'Caregive Support Programs Participation',
'Other Caregiver Engagement',
'Continue Participation',
'Caregiver Community Connectedness']
tdf = conditionalDrop(toDrop, tdf)
sdf = conditionalDrop(toDrop, sdf)

In [104]:
toDrop = ['Race:', 'Sex', 'Age', 'Time Providing Care', 'Use of Video Conferencing','Date (Responses to recipient forms)', 'Recipient', 'Origin (Responses to recipient forms)', 'Primary key', 'Conditional: Portal Engagement Explanation', 'Consent',]
tdf = conditionalDrop(toDrop, tdf)
sdf = conditionalDrop(toDrop, sdf)

In [105]:
tdf.drop(list(tdf.filter(regex = 'Chronic Conditions')), axis = 1, inplace = True)
tdf.drop(list(tdf.filter(regex = 'Constituent Type')), axis = 1, inplace = True)
tdf = tdf.drop(columns = ['Score (Responses to recipient forms)'])
sdf.drop(list(sdf.filter(regex = 'Chronic Conditions')), axis = 1, inplace = True)
sdf.drop(list(sdf.filter(regex = 'Constituent Type')), axis = 1, inplace = True)
sdf = sdf.drop(columns = ['Score (Responses to recipient forms)'])

In [106]:
datax = tdf.apply(f, axis = 0)
datax =datax.sample(frac=1).reset_index(drop=True)
datay = sdf.apply(f, axis = 0)
datay =datay.sample(frac=1).reset_index(drop=True)

In [107]:
datax = datax.loc[(datax!=-9).all(axis=1)]
datax = datax.loc[(datax!=-10).all(axis=1)]
datay = datay.loc[(datay!=-9).all(axis=1)]
datay = datay.loc[(datay!=-10).all(axis=1)]

In [108]:
datax

,Portal Chronic Disease Management Help,SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),Feel Excluded,Difficult to make friends,Need Company,Unable To Communicate,Feel Completely Alone,Waiting for Contact,No One Understands Me,Cant Tolerate Being Alone,No One To Talk To,Unhappy Alone,Portal Wellbeing Influence,Portal Engagement Reinforcement,Portal Engagement,Portal Experience,Relationship Impact,Concern About Future,Competing Demand,Demands Strength,Health Change,Financial Decrease,Same Self,Desire To Escape,Physical Exhaustion,Life Satisfaction,Motivation,Depression,Sense of Wellbeing,Sense of Hope,MVCN Support Programs and Services,MVCN Volunteer,MVCN Program Participation
0,2,1,1,1,0,0,1,1,0,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,2,2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-1,-1,2,2,1,-1,1
1,0,1,1,1,0,1,1,0,0,0,1,1,-1,-1,-1,-1,1,-1,1,1,1,1,0,0,0,1,2,1,1,0,1,1,0,0,-1,-1,1,1,-1,-1,1
2,2,1,1,1,0,0,1,1,0,0,1,2,2,1,1,1,1,1,1,2,1,2,2,2,2,1,2,1,1,0,2,2,2,2,1,1,1,1,-1,-1,-1
3,0,1,0,0,0,0,1,0,0,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,1,-1,-1
4,0,0,1,0,0,0,1,1,0,0,-2,-2,-2,-2,1,1,-1,-2,-1,1,2,1,1,1,0,0,1,2,1,2,1,-1,1,-2,-1,1,1,2,1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,0,1,0,1,0,1,0,0,0,0,-2,-2,-2,-2,-2,-2,-1,-2,-2,-2,2,2,0,2,0,-2,-2,-2,0,1,1,1,2,-2,1,1,1,1,-1,-1,-1
308,1,1,1,0,0,1,1,1,1,0,1,2,-1,-1,-1,-1,1,-1,1,1,0,1,1,1,1,0,1,2,1,1,1,1,2,0,1,1,2,2,1,-1,-1
309,2,1,1,1,0,1,1,1,0,0,1,1,-1,-1,1,1,-1,-1,1,1,2,2,2,2,0,2,0,1,1,0,-2,-2,0,-1,-1,-1,2,2,1,-1,-1
310,1,1,1,1,0,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,2,2,2,0,0,0,0,0,0,2,2,2,0,1,1,1,0,1,1,1


In [109]:
datay

,MVCN Program Participation,MVCN Volunteer,MVCN Support Programs and Services,Sense of Hope,Sense of Wellbeing,Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help
1,1,-1,0,1,1,1,1,0,1,0,0,-1,-1,1,1,1,0,2,1,1,1,-1,-1,1,1,1,-1,1,1,1,1,1,1,1,0,0,0,0,0,1,1
2,1,-1,-1,0,1,1,-1,1,1,1,1,-1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,1,0,1
3,1,-1,1,2,2,-1,-1,0,1,1,1,0,1,-1,-1,0,-1,2,2,2,2,1,1,-1,1,-1,-1,1,1,2,1,1,1,1,1,0,1,1,1,0,2
4,1,-1,1,1,1,1,1,1,0,0,0,2,2,0,0,1,1,2,2,2,2,1,1,-1,1,1,1,1,1,-1,1,1,0,1,0,0,1,1,0,0,1
5,1,-1,1,2,2,1,1,-1,1,1,1,-1,1,0,0,-1,0,1,0,1,1,1,1,-1,1,1,1,-1,-1,1,1,1,1,1,0,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,1,-1,1,1,1,-1,1,2,2,0,2,2,2,2,2,2,2,2,1,2,2,-1,-1,-2,2,1,-1,-1,-1,1,1,0,0,0,1,0,0,0,0,1,1
305,1,0,1,1,1,-1,1,-2,-1,-1,0,-2,-2,-2,-1,-2,-1,2,1,1,1,1,1,-1,1,-1,-1,-1,-1,1,1,1,0,1,0,0,0,1,1,0,1
306,1,-1,0,2,2,1,1,-1,1,-1,1,1,-1,1,1,-1,1,1,2,2,2,1,1,1,1,1,1,1,1,-1,-1,1,1,1,0,0,1,1,1,0,2
307,1,-1,-1,1,1,1,1,1,1,1,1,-1,1,2,2,2,2,2,2,1,0,1,1,-1,1,2,1,1,1,2,2,1,1,1,0,0,1,1,1,0,0


In [114]:
dataz = pd.concat([datax, datay], ignore_index=True)

In [111]:
columns = ['MVCN Support Programs and Services','MVCN Volunteer','MVCN Program Participation']
for x in columns:
    a = datax[datax[x]==--1].shape[0]
    b = datax[datax[x]==-1].shape[0]
    c = datax[datax[x]==-0].shape[0]
    print(f'{x:50}  {a:5}  {b:5}  {c:5}  {a/2.65:.2f}  {b/2.65:.2f}  {c/2.65:.2f}')


MVCN Support Programs and Services                    108     96     61  40.75  36.23  23.02
MVCN Volunteer                                         25    218     22  9.43  82.26  8.30
MVCN Program Participation                            128     81     56  48.30  30.57  21.13


In [113]:
for x in columns:
    a = datay[datay[x]==--1].shape[0]
    b = datay[datay[x]==-1].shape[0]
    c = datay[datay[x]==-0].shape[0]
    print(f'{x:50}  {a:5}  {b:5}  {c:5}  {a/2.77:.2f}  {b/2.77:.2f}  {c/2.77:.2f}')


MVCN Support Programs and Services                    129     77     71  48.68  27.80  25.63
MVCN Volunteer                                         25    224     28  9.43  80.87  10.11
MVCN Program Participation                            154     80     43  58.11  28.88  15.52


In [115]:
for x in columns:
    a = dataz[dataz[x]==--1].shape[0]
    b = dataz[dataz[x]==-1].shape[0]
    c = dataz[dataz[x]==-0].shape[0]
    print(f'{x:50}  {a:5}  {b:5}  {c:5}  {a/5.42:.2f}  {b/5.42:.2f}  {c/5.42:.2f}')

MVCN Support Programs and Services                    237    173    132  43.73  31.92  24.35
MVCN Volunteer                                         50    442     50  9.23  81.55  9.23
MVCN Program Participation                            282    161     99  52.03  29.70  18.27


In [131]:
for x in columns:
    a = stats(datax[datax[x] == -1], datax[datax[x] == 1])
    display(a[:19])
    display(a[19:])

,Item,P-Value,difference
0,Portal Chronic Disease Management Help,0.001881,none
1,SMS/text (Connection Technology Use),0.453801,none
2,Phone (Connection Technology Use),0.179526,none
3,Video Portal (Connection Technology Use),0.177946,none
4,Other (Connection Technology Use),0.240262,none
5,Caregivers (Portal Video Use),0.001892,none
6,Family members (Portal Video Use),0.431740,none
7,Friends (Portal Video Use),0.396346,none
8,Others (Portal Video Use),0.385307,none
9,Not using Portal video (Portal Video Use),0.011866,none


,Item,P-Value,difference
19,Unhappy Alone,2.282585e-02,none
20,Portal Wellbeing Influence,3.179914e-04,larger
21,Portal Engagement Reinforcement,2.255181e-05,larger
22,Portal Engagement,2.292786e-03,none
23,Portal Experience,6.287260e-02,none
24,Relationship Impact,3.756236e-01,none
25,Concern About Future,3.321982e-01,none
26,Competing Demand,4.369136e-01,none
27,Demands Strength,4.776530e-01,none
28,Health Change,3.888185e-01,none


,Item,P-Value,difference
0,Portal Chronic Disease Management Help,1.179765e-02,none
1,SMS/text (Connection Technology Use),1.594959e-01,none
2,Phone (Connection Technology Use),4.956236e-01,none
3,Video Portal (Connection Technology Use),2.005361e-03,none
4,Other (Connection Technology Use),6.127655e-07,smaller
5,Caregivers (Portal Video Use),2.083653e-04,larger
6,Family members (Portal Video Use),1.462516e-01,none
7,Friends (Portal Video Use),7.512457e-02,none
8,Others (Portal Video Use),1.883428e-01,none
9,Not using Portal video (Portal Video Use),1.509010e-01,none


,Item,P-Value,difference
19,Unhappy Alone,8.317662e-03,none
20,Portal Wellbeing Influence,3.365055e-01,none
21,Portal Engagement Reinforcement,5.652273e-02,none
22,Portal Engagement,3.347129e-02,none
23,Portal Experience,1.858111e-01,none
24,Relationship Impact,2.094232e-01,none
25,Concern About Future,5.000000e-01,none
26,Competing Demand,3.997754e-01,none
27,Demands Strength,5.214790e-02,none
28,Health Change,1.905759e-01,none


,Item,P-Value,difference
0,Portal Chronic Disease Management Help,0.007345,none
1,SMS/text (Connection Technology Use),0.247815,none
2,Phone (Connection Technology Use),0.126517,none
3,Video Portal (Connection Technology Use),0.047903,none
4,Other (Connection Technology Use),0.145960,none
5,Caregivers (Portal Video Use),0.028791,none
6,Family members (Portal Video Use),0.343488,none
7,Friends (Portal Video Use),0.169323,none
8,Others (Portal Video Use),0.031343,none
9,Not using Portal video (Portal Video Use),0.060129,none


,Item,P-Value,difference
19,Unhappy Alone,2.069455e-01,none
20,Portal Wellbeing Influence,8.332263e-02,none
21,Portal Engagement Reinforcement,4.423432e-02,none
22,Portal Engagement,2.515904e-01,none
23,Portal Experience,4.436720e-01,none
24,Relationship Impact,1.589518e-02,none
25,Concern About Future,3.585163e-01,none
26,Competing Demand,1.005655e-01,none
27,Demands Strength,4.734062e-01,none
28,Health Change,2.729921e-01,none


In [116]:
for x in columns:
    a = stats(datay[datay[x] == -1], datay[datay[x] == 1])
    display(a[:19])
    display(a[19:])

,Item,P-Value,difference
0,MVCN Program Participation,1.134738e-11,larger
1,MVCN Volunteer,6.505138e-05,larger
2,MVCN Support Programs and Services,0.000000e+00,larger
3,Sense of Hope,1.758374e-10,larger
4,Sense of Wellbeing,4.080771e-08,larger
5,Depression,8.728257e-02,none
6,Motivation,1.255042e-01,none
7,Life Satisfaction,3.051546e-02,none
8,Physical Exhaustion,9.306037e-02,none
9,Desire To Escape,5.632787e-02,none


,Item,P-Value,difference
19,Portal Engagement Reinforcement,4.245981e-05,larger
20,Portal Wellbeing Influence,3.325444e-05,larger
21,Unhappy Alone,1.555133e-01,none
22,No One To Talk To,1.481850e-01,none
23,Cant Tolerate Being Alone,2.304901e-01,none
24,No One Understands Me,2.799123e-01,none
25,Waiting for Contact,4.278663e-02,none
26,Feel Completely Alone,1.874403e-02,none
27,Unable To Communicate,6.421532e-02,none
28,Need Company,1.237366e-02,none


,Item,P-Value,difference
0,MVCN Program Participation,2.330418e-03,none
1,MVCN Volunteer,0.000000e+00,larger
2,MVCN Support Programs and Services,4.387637e-06,larger
3,Sense of Hope,3.953058e-03,none
4,Sense of Wellbeing,1.278160e-03,larger
5,Depression,2.589761e-01,none
6,Motivation,4.083651e-01,none
7,Life Satisfaction,3.871176e-01,none
8,Physical Exhaustion,1.881762e-01,none
9,Desire To Escape,4.057043e-01,none


,Item,P-Value,difference
19,Portal Engagement Reinforcement,7.563782e-02,none
20,Portal Wellbeing Influence,9.405902e-03,none
21,Unhappy Alone,2.351754e-01,none
22,No One To Talk To,1.915674e-01,none
23,Cant Tolerate Being Alone,4.526921e-01,none
24,No One Understands Me,3.603130e-02,none
25,Waiting for Contact,2.557558e-01,none
26,Feel Completely Alone,3.770843e-01,none
27,Unable To Communicate,1.407086e-01,none
28,Need Company,3.953767e-01,none


,Item,P-Value,difference
0,MVCN Program Participation,0.000000e+00,larger
1,MVCN Volunteer,6.600505e-03,none
2,MVCN Support Programs and Services,6.682631e-13,larger
3,Sense of Hope,2.413565e-07,larger
4,Sense of Wellbeing,1.925151e-06,larger
5,Depression,2.447581e-01,none
6,Motivation,9.782318e-02,none
7,Life Satisfaction,3.603583e-01,none
8,Physical Exhaustion,5.726600e-02,none
9,Desire To Escape,1.506040e-01,none


,Item,P-Value,difference
19,Portal Engagement Reinforcement,4.151495e-03,none
20,Portal Wellbeing Influence,2.606307e-02,none
21,Unhappy Alone,2.924429e-01,none
22,No One To Talk To,1.649239e-01,none
23,Cant Tolerate Being Alone,2.742140e-01,none
24,No One Understands Me,2.818622e-01,none
25,Waiting for Contact,3.509983e-01,none
26,Feel Completely Alone,3.866089e-01,none
27,Unable To Communicate,1.630767e-01,none
28,Need Company,3.900940e-01,none


In [117]:
for x in columns:
    a = stats(dataz[dataz[x] == -1], dataz[dataz[x] == 1])
    display(a[:19])
    display(a[19:])

,Item,P-Value,difference
0,Portal Chronic Disease Management Help,3.621441e-06,larger
1,SMS/text (Connection Technology Use),8.346720e-02,none
2,Phone (Connection Technology Use),4.415266e-01,none
3,Video Portal (Connection Technology Use),5.112843e-04,larger
4,Other (Connection Technology Use),3.381168e-01,none
5,Caregivers (Portal Video Use),2.142592e-05,larger
6,Family members (Portal Video Use),1.901266e-01,none
7,Friends (Portal Video Use),2.128208e-01,none
8,Others (Portal Video Use),1.294789e-01,none
9,Not using Portal video (Portal Video Use),3.962584e-04,smaller


,Item,P-Value,difference
19,Unhappy Alone,1.540645e-02,none
20,Portal Wellbeing Influence,6.342627e-08,larger
21,Portal Engagement Reinforcement,9.801513e-09,larger
22,Portal Engagement,1.283105e-06,larger
23,Portal Experience,1.013469e-04,larger
24,Relationship Impact,2.705413e-01,none
25,Concern About Future,3.663687e-01,none
26,Competing Demand,1.549124e-01,none
27,Demands Strength,1.101773e-01,none
28,Health Change,1.747961e-01,none


,Item,P-Value,difference
0,Portal Chronic Disease Management Help,1.082461e-03,larger
1,SMS/text (Connection Technology Use),3.209943e-01,none
2,Phone (Connection Technology Use),4.174224e-01,none
3,Video Portal (Connection Technology Use),1.316258e-03,none
4,Other (Connection Technology Use),3.976191e-05,smaller
5,Caregivers (Portal Video Use),6.165733e-07,larger
6,Family members (Portal Video Use),1.811009e-01,none
7,Friends (Portal Video Use),5.566074e-02,none
8,Others (Portal Video Use),3.443365e-01,none
9,Not using Portal video (Portal Video Use),6.078516e-02,none


,Item,P-Value,difference
19,Unhappy Alone,1.499260e-02,none
20,Portal Wellbeing Influence,3.693025e-02,none
21,Portal Engagement Reinforcement,1.579133e-02,none
22,Portal Engagement,1.699270e-01,none
23,Portal Experience,1.485931e-01,none
24,Relationship Impact,1.034737e-01,none
25,Concern About Future,4.542756e-01,none
26,Competing Demand,2.536800e-01,none
27,Demands Strength,8.091529e-03,none
28,Health Change,1.058931e-01,none


,Item,P-Value,difference
0,Portal Chronic Disease Management Help,2.925189e-04,larger
1,SMS/text (Connection Technology Use),4.327004e-02,none
2,Phone (Connection Technology Use),3.571051e-01,none
3,Video Portal (Connection Technology Use),2.462200e-03,none
4,Other (Connection Technology Use),1.473668e-01,none
5,Caregivers (Portal Video Use),6.634084e-07,larger
6,Family members (Portal Video Use),1.543440e-01,none
7,Friends (Portal Video Use),3.831304e-01,none
8,Others (Portal Video Use),7.137915e-04,larger
9,Not using Portal video (Portal Video Use),4.347404e-04,smaller


,Item,P-Value,difference
19,Unhappy Alone,1.588132e-01,none
20,Portal Wellbeing Influence,8.318422e-03,none
21,Portal Engagement Reinforcement,1.061846e-03,larger
22,Portal Engagement,1.190778e-02,none
23,Portal Experience,1.780419e-02,none
24,Relationship Impact,3.749140e-04,larger
25,Concern About Future,8.675208e-02,none
26,Competing Demand,1.194513e-02,none
27,Demands Strength,3.060531e-01,none
28,Health Change,1.016027e-01,none


##Ignore everything past this

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
c1data['Depression'].shape[0]

In [ ]:
sum(c1data['Depression'].value_counts().tolist())

In [ ]:

def graph1(labels, values, vlabels, xlabel, ylabel, title):

    x = np.arange(len(labels))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots()
    ax.bar(x, values[0], width, label= vlabels[0])

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    fig.tight_layout()

    plt.show()

def graph2(labels, values, vlabels, xlabel, ylabel, title):

    x = np.arange(len(labels))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots()
    ax.bar(x - width/2, values[0], width, label= vlabels[0])
    ax.bar(x + width/2, values[1], width, label= vlabels[1])

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation = 15)
    ax.legend()

    fig.tight_layout()

    plt.show()

def graph3(labels, values, vlabels, xlabel, ylabel, title):

    x = np.arange(len(labels))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots()
    ax.bar(x - width, values[0], width, label= vlabels[0])
    ax.bar(x, values[1], width, label= vlabels[1])
    ax.bar(x + width, values[2], width, label= vlabels[2])

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    fig.tight_layout()

    plt.show()

In [ ]:
a = [1, 2, 3]

In [ ]:
np.array(a) / 2


In [ ]:
labels = ['Caretaker (Any)', 'Military Caregiver', 'Veteran Caregiver', 'Veterans', 'Neither']
values1 = [429, 11, 417, 294, 5]
values2 = [399, 9, 376, 241, 5]
graph2(labels, [np.array(values1)/702, np.array(values2)/605], ['Survey 1', 'Survey 2'], 'Role', 'Frequency', 'Relative Role Distribution')
graph2(labels, [np.array(values1), np.array(values2)], ['Survey 1', 'Survey 2'], 'Role', 'Frequency', 'Role Distribution')

In [ ]:
labels = ['Depressed', 'Not Depressed']
graph1(labels, [c1data['Depression'].value_counts().tolist()], ['c1'], '', 'Frequency', 'Depression Distribution')

In [ ]:
pd.set_option('display.max_rows', 100)
data.sort_values(by=['survey'])

In [ ]:
#toDrop2 = ['Depression', 'Motivation', 'Physical Exhaustion', 'Desire To Escape', 'Same Self', 'Financial Decrease', 'Health Change', 'Demands Strength', 'Competing Demand', 'Concern About Future', 'Relationship Impact', 'Unhappy Alone', 'No One To Talk To', 'Cant Tolerate Being Alone', 'No One Understands Me', 'Waiting for Contact', 'Feel Completely Alone', 'Unable To Communicate', 'Need Company', 'Difficult to make friends', 'Feel Excluded']
toDrop3 = ['Score (Responses to recipient forms)']
data1 = data.drop(columns = toDrop2)
data1 = data.drop(columns = toDrop3)
data1.shape

In [ ]:
test1 = data[data['survey']==1]['Use of Video Conferencing']
test2 = data[data['survey']==2]['Use of Video Conferencing']
print(test1.sum()/test1.shape[0])
print(test2.sum()/test2.shape[0])

In [ ]:
train_size = int(data1.shape[0]*0.8)
test_size = int(data1.shape[0]*0.2)

In [ ]:
#creates Bayesian model using data

dtrain = data1[:train_size]
dtest = data1[:-test_size]

In [ ]:
model = BayesianNetwork.from_samples(dtrain)

In [ ]:
blanked = dtest.copy()
blanked['Score (Responses to recipient forms)'] = np.full(blanked.shape[0], None)
prediction = model.predict(blanked.values.tolist())
mean = dtest["Score (Responses to recipient forms)"].mean()
rss = 0
tss = 0
correct = 0
for i in range (0, dtest.shape[0]):
    yBar = prediction[i][0]
    
    yActual = dtest['Score (Responses to recipient forms)'][i]
    
    if (yBar == yActual):
        correct+=1
    
    rss += (yActual-yBar)**2
    tss += (yActual-mean)**2

percent = float(correct) / dtest.shape[0]
r2 = 1-float(rss)/tss
print("Accuracy: " + (str)(percent))
print("r^2: " + (str)(r2))


from sklearn.metrics import precision_score, recall_score, accuracy_score, r2_score

labels = [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
y_true = dtest['Score (Responses to recipient forms)']
y_pred = [row[0] for row in prediction]

precision = precision_score(y_true, y_pred, labels=labels, average='micro')
recall = recall_score(y_true, y_pred, labels=labels, average='micro')
accuracy = accuracy_score(y_true, y_pred)
# Need to make y_true shape (samples, classes)

#y_true = y_true.values.reshape((y_true.shape[0],len(labels)))
#y_pred = y_pred.reshape((y_pred.shape[0],len(labels)))

r_squared = r2_score(y_true, y_pred)
#auc = roc_auc_score(y_true, y_pred)

print('Accuracy: %.3f' % accuracy)
print('R^2: %.3f' % r_squared)
print('Precision: %.3f' % precision)
print('Recall: %.3f' % recall)
#print('ROC AUC: %.3f' % auc)

In [ ]:
blanked = dtest.copy()
blanked['Life Satisfaction'] = np.full(blanked.shape[0], None)
prediction = model.predict(blanked.values.tolist())
mean = dtest["Life Satisfaction"].mean()
rss = 0
tss = 0
correct = 0
for i in range (0, dtest.shape[0]):
    yBar = prediction[i][0]
    
    yActual = dtest['Life Satisfaction'][i]
    print(yBar)
    if (yBar == yActual):
        correct+=1
    
    rss += (yActual-yBar)**2
    tss += (yActual-mean)**2

percent = float(correct) / dtest.shape[0]
r2 = 1-float(rss)/tss
print("Accuracy: " + (str)(percent))
print("r^2: " + (str)(r2))


from sklearn.metrics import precision_score, recall_score, accuracy_score, r2_score

labels = [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
y_true = dtest['Life Satisfaction']
y_pred = [row[0] for row in prediction]

precision = precision_score(y_true, y_pred, labels=labels, average='micro')
recall = recall_score(y_true, y_pred, labels=labels, average='micro')
accuracy = accuracy_score(y_true, y_pred)
# Need to make y_true shape (samples, classes)

#y_true = y_true.values.reshape((y_true.shape[0],len(labels)))
#y_pred = y_pred.reshape((y_pred.shape[0],len(labels)))

r_squared = r2_score(y_true, y_pred)
#auc = roc_auc_score(y_true, y_pred)

print('Accuracy: %.3f' % accuracy)
print('R^2: %.3f' % r_squared)
print('Precision: %.3f' % precision)
print('Recall: %.3f' % recall)
#print('ROC AUC: %.3f' % auc)

In [ ]:
from pymc3.glm import GLM

with pm.Model() as model_glm:
    GLM.from_formula("Life_Satisfaction ~ Caregiver_for_Veteran + Other_caregiver + Veteran + Other + Hypertension_high_blood_pressure_Chronic_Conditions + Hyperlipidemia_high_blood_cholesterol_or_triglyceride_levels_Chronic_Conditions + Allergies + sinusitis_and_other_upper_respiratory_conditions_Chronic_Conditions + Arthritis_Chronic_Conditions + Mood_Disorder_depression_and_bipolar_disorder_Chronic_Conditions + Diabetes_Type1_and_Type_2_Chronic_Conditions + Anxiety_Disorder_Chronic_Conditions + Asthma_Chronic_Conditions + Coronary_artery_disease_includes_myocardial_infarction_heart_attack_Chronic_Conditions + Thyroid_disorder_Chronic_Conditions + Chronic_obstructive_lung_disease_and_bronchiectasis_Chronic_Condition + Traumatic_Brain_Injury__TBI_Chronic_Conditions + Post_Traumatic_Stress_Disorder_PTSD_Chronic_Conditions + None_of_the_above__Chronic_Conditions + Caregivers_Portal_Video_Use + Family_members__Portal_Video_Us + Friends_Portal_Video_Use + Others_Portal_Video_Use + Not_using_Portal_video_Portal_Video_Use + Portal_Chronic_Disease_Management_Help + SMS_text_Connection_Technology_Use + Phone_Connection_Technology_Use + Video_Portal_Connection_Technology_Use + Other_Connection_Technology_Use + before", dtrain)
    trace = pm.sample()

In [ ]:
with model_glm:
    axes_arr = az.plot_trace(trace)
plt.draw()

In [ ]:
blanked = dtest.copy(dtest)
blanked['Life_Satisfaction'] = np.full(blanked.shape[0])
prediction = model.predict(blanked)
mean = dtest["weight"].mean()
rss = 0
tss = 0
correct = 0
for i in range (0, dtest.shape[0]):
    yBar = prediction['Life_Satisfaction'][i]
    yActual = dtest['Life_Satisfaction'][i]
    if (yBar == yActual):
        correct+=1
    rss += (yActual-yBar)**2
    tss += (yActual-mean)**2
percent = (float)correct/200
r2 = 1-(float)rss/tss
print("Accuracy: " + (str)(percent))
print("r^2: " + (str)(r2))

from sklearn.metrics import precision_score, recall_score, accuracy_score, r2_score

labels = [-2, -1, 0, 1, 2]
y_true = dtest['Life_Satisfaction']
y_pred = prediction['Life_Satisfaction']

precision = precision_score(y_true, y_pred, labels=labels, average='micro')
recall = recall_score(y_true, y_pred, labels=labels, average='micro')
accuracy = accuracy_score(y_true, y_pred)
# Need to make y_true shape (samples, classes)

y_true = y_true.reshape((y_true.shape[0],len(labels)))
y_pred = y_pred.reshape((y_pred.shape[0],len(labels)))

r_squared = r2_score(y_true, y_pred)
auc = roc_auc_score(y_true, y_pred)

print('Accuracy: %.3f' % accuracy)
print('R^2: %.3f' % r_squared)
print('Precision: %.3f' % precision)
print('Recall: %.3f' % recall)
print('ROC AUC: %.3f' % auc)